In [1]:
from simple.ohlc import debounce, midPrice, TTrade
from simple.plotly import chartFigure
from simple.funcs import symlog
import numpy as np

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)[:3300]
T[-5:]

rec.array([('2021-06-01T00:06:08.611000', '2021-06-01T00:06:08.653000', 37615.,  -5649., nan),
           ('2021-06-01T00:06:08.611000', '2021-06-01T00:06:08.653000', 37615.,   -500., nan),
           ('2021-06-01T00:06:08.611000', '2021-06-01T00:06:08.653000', 37615.,   -800., nan),
           ('2021-06-01T00:06:08.611000', '2021-06-01T00:06:08.653000', 37615., -50000., nan),
           ('2021-06-01T00:06:08.611000', '2021-06-01T00:06:08.653000', 37615., -50000., nan)],
          dtype=[('DateTimeA', '<M8[us]'), ('LocalTimeA', '<M8[us]'), ('PriceA', '<f8'), ('VolumeA', '<f8'), ('OpenIntA', '<f8')])

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]

In [4]:
C = debounce(T)
C[-5:]

rec.array([('2021-06-01T00:06:08.594000', 726, 37620.25, 1999, 116000., 0., -116000., 3, 0, 3),
           ('2021-06-01T00:06:08.595999', 727, 37618.75,    0,    377., 0.,    -377., 1, 0, 1),
           ('2021-06-01T00:06:08.595999', 728, 37617.75,    0,   5000., 0.,   -5000., 1, 0, 1),
           ('2021-06-01T00:06:08.595999', 729, 37616.25,    0,  29600., 0.,  -29600., 1, 0, 1),
           ('2021-06-01T00:06:08.611000', 730, 37615.25,    0, 125949., 0., -125949., 6, 0, 6)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [5]:
len(T), len(C)

(3300, 731)

In [6]:
# indexed chart
fig = chartFigure(height=400,
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.2),
    Debounce=dict(mode='markers+lines', x=C.DT, y=C.Price, color='black', connectgaps=True, opacity=0.7)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2021-06-01T00:06:08.594000', 726, 37620.25, 1999, -116000., 0., -116000., 3, 0, 3),
           ('2021-06-01T00:06:08.595999', 727, 37618.75,    0,    -377., 0.,    -377., 1, 0, 1),
           ('2021-06-01T00:06:08.595999', 728, 37617.75,    0,   -5000., 0.,   -5000., 1, 0, 1),
           ('2021-06-01T00:06:08.595999', 729, 37616.25,    0,  -29600., 0.,  -29600., 1, 0, 1),
           ('2021-06-01T00:06:08.611000', 730, 37615.25,    0, -125949., 0., -125949., 6, 0, 6)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [8]:
# indexed chart without bidask
fig = chartFigure(height=400,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.4),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.4),
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.4,
    …

In [9]:
T = np.zeros(len(C), dtype=TTrade).view(np.recarray)
T.DateTimeA = C.DT
T.PriceA = np.round(C.Price, -1)
T.VolumeA = C.Volume
T[-5:]

rec.array([('2021-06-01T00:06:08.594000', '1970-01-01T00:00:00.000000', 37620., -116000., 0.),
           ('2021-06-01T00:06:08.595999', '1970-01-01T00:00:00.000000', 37620.,    -377., 0.),
           ('2021-06-01T00:06:08.595999', '1970-01-01T00:00:00.000000', 37620.,   -5000., 0.),
           ('2021-06-01T00:06:08.595999', '1970-01-01T00:00:00.000000', 37620.,  -29600., 0.),
           ('2021-06-01T00:06:08.611000', '1970-01-01T00:00:00.000000', 37620., -125949., 0.)],
          dtype=[('DateTimeA', '<M8[us]'), ('LocalTimeA', '<M8[us]'), ('PriceA', '<f8'), ('VolumeA', '<f8'), ('OpenIntA', '<f8')])

In [10]:
C = debounce(T)
len(T), len(C)

(731, 197)

In [11]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2021-06-01T00:05:47.966000', 191, 37655.,       0, -2.00000e+01, 0., -2.00000e+01, 1, 0, 1),
           ('2021-06-01T00:05:49.479000', 193, 37655.,   24999, -6.23000e+02, 0., -6.23000e+02, 2, 0, 2),
           ('2021-06-01T00:05:49.529000', 194, 37645., 8350999, -7.30543e+05, 0., -7.30543e+05, 5, 0, 5),
           ('2021-06-01T00:05:57.900000', 195, 37635.,  722000, -2.13910e+04, 0., -2.13910e+04, 2, 0, 2),
           ('2021-06-01T00:06:08.594000', 196, 37625.,   17000, -2.98477e+05, 0., -2.98477e+05, 6, 0, 6)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [12]:
# indexed chart without bidask
fig = chartFigure(height=400,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.4),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.4),
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.4,
    …